# Question 2.1 

In [1]:
# Data Handling
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# Progress Bar
from tqdm import tqdm



In [2]:
static_variables = ['RecordID', 'Age', 'Gender', 'Height', 'ICUType', 'Weight']

static_variables.remove('ICUType')

static_variables_we_want = ['Age', 'Gender', 'Height', 'Weight']
all_variables = ['Weight', 'Age', 'TroponinI', 'DiasABP', 'MechVent', 'HCO3', 'Cholesterol', 'HCT', 'SaO2', 'WBC', 'SysABP', 'Urine', 'ICUType', 'Gender', 'ALP', 'Creatinine', 'K', 'AST', 'Glucose', 'RespRate', 'MAP', 'FiO2', 'BUN', 'Na', 'Bilirubin', 'TroponinT', 'PaCO2', 'Height', 'GCS', 'HR', 'pH', 'PaO2', 'Lactate', 'ALT', 'NISysABP', 'RecordID', 'Platelets', 'Temp', 'Mg', 'NIDiasABP', 'Albumin', 'NIMAP']
dyn_variables = [x for x in all_variables if x not in static_variables]
dyn_variables.remove('ICUType')
dyn_variables.append('Weight_VAR')
len(dyn_variables), len(static_variables_we_want)

initial_column_lists = static_variables_we_want + dyn_variables

In [3]:
# import parquet file 
df_a =pd.read_parquet('data/processed_raw_data_set-a_1.parquet', engine='pyarrow')
df_b =pd.read_parquet('data/processed_raw_data_set-b_1.parquet', engine='pyarrow')
df_c =pd.read_parquet('data/processed_raw_data_set-c_1.parquet', engine='pyarrow')

drop_ICUType = True 
if drop_ICUType:
    df_a = df_a.drop(columns=['ICUType'])
    df_b = df_b.drop(columns=['ICUType'])
    df_c = df_c.drop(columns=['ICUType'])


#  drop Time variable in df_a
if 'Time' in df_a.columns:
    df_a = df_a.drop(columns=['Time'])
    df_b = df_b.drop(columns=['Time'])
    df_c = df_c.drop(columns=['Time'])


### Computing features vectors of our patient

Instead of working on the table where the missing values had already been filled, i prefer working on the not filled table because otherwise filled values would be taken into the mean and might flatten patient with lots of missing values. Then I compute the mean of variables for eahc patient over the 49 timestamps. 

Then i compute the median on the resulting table to fill the missing values

In [4]:
len(static_variables_we_want) , len(dyn_variables), len(static_variables_we_want) + len(dyn_variables) 

(4, 37, 41)

In [5]:
# we define aggregation functions
agg_funcs = {col: ['mean'] for col in dyn_variables}  

agg_funcs['RecordID'] = 'first'  # Keep RecordID
for stat_var in static_variables_we_want:
    if stat_var in df_a.columns:
        agg_funcs[stat_var] = 'first'  # Keep static variables

df_a_agg = df_a.groupby('RecordID').agg(agg_funcs)

df_a_agg.columns = ['-'.join(col).strip() for col in df_a_agg.columns.values]
df_a_agg.columns = [col.split('-')[0] for col in df_a_agg.columns]

In [6]:
df_a_agg

,TroponinI,DiasABP,MechVent,HCO3,Cholesterol,HCT,SaO2,WBC,SysABP,Urine,...,Mg,NIDiasABP,Albumin,NIMAP,Weight_VAR,RecordID,Age,Gender,Height,Weight
RecordID,,,,,,,,,,,,,,,,,,,,,
132539.0,NaN,NaN,NaN,27.000000,NaN,32.500000,NaN,10.300000,NaN,135.416667,...,1.700000,49.803030,NaN,70.949848,NaN,132539.0,54.0,0.0,NaN,NaN
132540.0,NaN,58.448148,1.0,22.333333,NaN,28.655556,96.833333,11.266667,116.051852,126.816327,...,2.300000,57.111111,NaN,75.971667,80.680000,132540.0,76.0,1.0,175.3,76.0
132541.0,NaN,67.666667,1.0,25.000000,NaN,28.460000,95.000000,4.700000,126.500000,106.729167,...,1.720000,78.963542,2.5,96.780573,56.700000,132541.0,44.0,0.0,NaN,56.7
132543.0,NaN,NaN,NaN,27.666667,NaN,37.442857,NaN,9.400000,NaN,66.836735,...,2.033333,63.853261,4.4,82.712799,84.600000,132543.0,68.0,1.0,180.3,84.6
132545.0,NaN,NaN,NaN,19.000000,NaN,29.550000,NaN,4.300000,NaN,50.234043,...,1.550000,45.962500,3.3,74.911875,NaN,132545.0,88.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142665.0,NaN,69.187943,1.0,21.500000,NaN,29.233333,NaN,14.500000,117.000000,40.408163,...,1.600000,62.966667,NaN,79.144333,87.000000,142665.0,70.0,0.0,NaN,87.0
142667.0,NaN,NaN,NaN,23.800000,117.0,31.333333,NaN,4.733333,NaN,173.148148,...,1.440000,58.476190,NaN,81.628571,166.400000,142667.0,25.0,1.0,NaN,166.4
142670.0,NaN,72.125000,1.0,24.750000,NaN,38.750000,NaN,11.066667,122.625000,93.177778,...,2.400000,60.032051,NaN,79.549167,109.000000,142670.0,44.0,1.0,NaN,109.0


In [7]:
# same for df_b 
agg_funcs = {col: ['mean'] for col in dyn_variables}
agg_funcs['RecordID'] = 'first'  # Keep RecordID
for stat_var in static_variables_we_want:
    if stat_var in df_b.columns:
        agg_funcs[stat_var] = 'first'  # Keep static variables

# Compute mean and std in one go
df_b_agg = df_b.groupby('RecordID').agg(agg_funcs)

df_b_agg.columns = ['-'.join(col).strip() for col in df_b_agg.columns.values]
df_b_agg.columns = [col.split('-')[0] for col in df_b_agg.columns]

In [8]:
# same for df_c
agg_funcs = {col: ['mean'] for col in dyn_variables}
agg_funcs['RecordID'] = 'first'  # Keep RecordID
for stat_var in static_variables_we_want:
    if stat_var in df_c.columns:
        agg_funcs[stat_var] = 'first'  # Keep static variables

# Compute mean and std in one go
df_c_agg = df_c.groupby('RecordID').agg(agg_funcs)
df_c_agg.columns = ['-'.join(col).strip() for col in df_c_agg.columns.values]
df_c_agg.columns = [col.split('-')[0] for col in df_c_agg.columns]

In [9]:
# compute median of df_a 
df_a_agg_median = df_a_agg.median()



In [10]:
# fill missing values with median
df_a_agg.fillna(df_a_agg_median, inplace=True)

df_b_agg.fillna(df_a_agg_median, inplace=True)
df_c_agg.fillna(df_a_agg_median, inplace=True)

In [11]:
assert df_a_agg.isnull().sum().sum() == 0
assert df_b_agg.isnull().sum().sum() == 0
assert df_c_agg.isnull().sum().sum() == 0

In [12]:
df_a_agg

,TroponinI,DiasABP,MechVent,HCO3,Cholesterol,HCT,SaO2,WBC,SysABP,Urine,...,Mg,NIDiasABP,Albumin,NIMAP,Weight_VAR,RecordID,Age,Gender,Height,Weight
RecordID,,,,,,,,,,,,,,,,,,,,,
132539.0,2.133333,58.847518,1.0,27.000000,153.0,32.500000,97.250000,10.300000,117.994866,135.416667,...,1.700000,49.803030,3.0,70.949848,80.121087,132539.0,54.0,0.0,170.2,78.6
132540.0,2.133333,58.448148,1.0,22.333333,153.0,28.655556,96.833333,11.266667,116.051852,126.816327,...,2.300000,57.111111,3.0,75.971667,80.680000,132540.0,76.0,1.0,175.3,76.0
132541.0,2.133333,67.666667,1.0,25.000000,153.0,28.460000,95.000000,4.700000,126.500000,106.729167,...,1.720000,78.963542,2.5,96.780573,56.700000,132541.0,44.0,0.0,170.2,56.7
132543.0,2.133333,58.847518,1.0,27.666667,153.0,37.442857,97.250000,9.400000,117.994866,66.836735,...,2.033333,63.853261,4.4,82.712799,84.600000,132543.0,68.0,1.0,180.3,84.6
132545.0,2.133333,58.847518,1.0,19.000000,153.0,29.550000,97.250000,4.300000,117.994866,50.234043,...,1.550000,45.962500,3.3,74.911875,80.121087,132545.0,88.0,0.0,170.2,78.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142665.0,2.133333,69.187943,1.0,21.500000,153.0,29.233333,97.250000,14.500000,117.000000,40.408163,...,1.600000,62.966667,3.0,79.144333,87.000000,142665.0,70.0,0.0,170.2,87.0
142667.0,2.133333,58.847518,1.0,23.800000,117.0,31.333333,97.250000,4.733333,117.994866,173.148148,...,1.440000,58.476190,3.0,81.628571,166.400000,142667.0,25.0,1.0,170.2,166.4
142670.0,2.133333,72.125000,1.0,24.750000,153.0,38.750000,97.250000,11.066667,122.625000,93.177778,...,2.400000,60.032051,3.0,79.549167,109.000000,142670.0,44.0,1.0,170.2,109.0


In [13]:
df_a_agg.to_parquet('data/set_a_for_q2_1.parquet', engine='pyarrow', index=False)
df_b_agg.to_parquet('data/set_b_for_q2_1.parquet', engine='pyarrow', index=False)
df_c_agg.to_parquet('data/set_c_for_q2_1.parquet', engine='pyarrow', index=False)